# Environment setup

In [1]:
import os

import gromacs

NOTE: Some configuration directories are not set up yet: 
	/home/nikitos/.gromacswrapper
	/home/nikitos/.gromacswrapper/qscripts
	/home/nikitos/.gromacswrapper/templates
NOTE: You can create the configuration file and directories with:
	>>> import gromacs
	>>> gromacs.config.setup()


All settings are made in the cell below. 

* `SIMULATION_DIRECTORY` - The directory within which all simulations will be run and all files will be saved;
* `MDP_FILES_DIRECTORY` - Directory with parameter files (`.mdp`) for molecular dynamics simulations;
* `PDB_STRUCTURE` - PDB file with antibody structure. This file should be in `SIMULATION_DIRECTORY`.

In [2]:
SIMULATION_DIRECTORY = 'remicade_humanization/remicade/'
MDP_FILES_DIRECTORY = 'mdp_files/'

PDB_STRUCTURE = '4g3y_Vdomens.pdb'

In [3]:
SIMULATION_DIRECTORY = os.path.abspath(SIMULATION_DIRECTORY)
MDP_FILES_DIRECTORY = os.path.abspath(MDP_FILES_DIRECTORY)

# Change the current working directory
os.chdir(SIMULATION_DIRECTORY)

# Print the current working directory
print(f'Current working directory: {os.getcwd()}')

Current working directory: /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade


# MD simulation

## Topology generation

Here we create files with topology (`.top`) and coordinates (`.gro`). Amber99sb force field is used, and tip3p for water model.

In [4]:
if not os.path.exists('structures/'):
    os.mkdir('structures')

In [5]:
gromacs.pdb2gmx(f=PDB_STRUCTURE,
                o='structures/processed.gro',
                p='topol.top',
                ff='amber99sb',
                water='tip3p'
                )

Using the Amber99sb force field in directory amber99sb.ff

going to rename amber99sb.ff/aminoacids.r2b

going to rename amber99sb.ff/dna.r2b

going to rename amber99sb.ff/rna.r2b
Reading 4g3y_Vdomens.pdb...
Read '', 1767 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 2 chains and 0 blocks of water and 227 residues with 1767 atoms

  chain  #res #atoms

  1 'H'   120    944  

  2 'L'   107    823  

All occupancies are one

Reading residue database... (Amber99sb)

Processing chain 1 'H' (944 atoms, 120 residues)

Identified residue GLU1 as a starting terminus.

Identified residue SER120 as a ending terminus.

Checking for duplicate atoms....

Generating any missing hydrogen atoms and/or adding termini.

Now there are 120 residues with 1840 atoms
Chain time...

Making bonds...

Number of bonds was 1864, now 1863

Generating angles, dihedrals and pairs...

Making cmap torsions...

There are 4881 dihedrals,  394 impropers, 3329 an

                      :-) GROMACS - gmx pdb2gmx, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx pdb2gmx -f 4g3y_Vdomens.pdb -o structures/processed.gro -p topol.top -ff amber99sb -water tip3p

Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/aminoacids.r2b
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/dna.r2b
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/rna.r2b
All occupancies are one
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/atomtypes.atp
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/aminoacids.rtp
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/dna.rtp
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb.ff/rna.rtp
Opening force fiel

(0, None, None)

## Defining unit cell and adding solvent

Let\`s define the box dimensions. We will use a rhombic dodecahedron box as the unit cell. After that we are filling the box with water

In [6]:
gromacs.editconf(f='structures/processed.gro',
                 c=True,
                 d=1.0,
                 bt='dodecahedron',
                 o='structures/newbox.gro'
                )

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3445 atoms
Volume: 2030.14 nm^3, corresponds to roughly 913500 electrons
No velocities found
    system size :  4.422  5.412  3.919 (nm)
    diameter    :  5.670               (nm)
    center      :  2.835 -0.531  1.231 (nm)
    box vectors : 15.366 15.366  9.928 (nm)
    box angles  :  90.00  90.00 120.00 (degrees)
    box volume  :2030.14               (nm^3)
    shift       :  2.917  6.283  1.481 (nm)
new center      :  5.752  5.752  2.712 (nm)
new box vectors :  7.670  7.670  7.670 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 319.06               (nm^3)


                      :-) GROMACS - gmx editconf, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx editconf -f structures/processed.gro -c -d 1.0 -bt dodecahedron -o structures/newbox.gro


GROMACS reminds you: "You're Insignificant" (Tricky)



(0, None, None)

In [7]:
gromacs.solvate(cp='structures/newbox.gro',
                cs='spc216.gro',
                o='structures/solvate.gro',
                p='topol.top'
               )


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for 9155 solvent molecules with resname (SOL) to topology file (topol.top)


                      :-) GROMACS - gmx solvate, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx solvate -cp structures/newbox.gro -cs spc216.gro -o structures/solvate.gro -p topol.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 5x5x3 boxes
Solvent box contains 35577 atoms in 11859 residues
Removed 4980 solvent atoms due to solvent-solvent overlap
Removed 3132 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
    SOL (   3 atoms):  9155 residues
Generated solvent containing 27465 atoms in 9155 residues
Writing generated configuration to structures/solvate.gro

Output configuration contains 30910 atoms in 9382 residues
Volume                 :     319.057 (nm

(0, None, None)

## Adding ions

Since life does not exist at a net charge, we must add ions to our system.

In [9]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'ions.mdp'),
               c='structures/solvate.gro',
               p='topol.top',
               o='ions.tpr'
              )

Setting the LD random seed to -92479582

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

Excluding 2 bonded neighbours molecule type 'SOL'
Analysing residue names:
There are:   227    Protein residues
There are:  9155      Water residues
Analysing Protein...

This run will generate roughly 3 Mb of data


                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/ions.mdp -c structures/solvate.gro -p topol.top -o ions.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/ions.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5

NOTE 2 [file topol.top, line 48]:
  System has non-zero total charge: -3.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.





(0, None, None)

In [10]:
gromacs.genion(s='ions.tpr',
               o='structures/solvate_ions.gro',
               p='topol.top',
               pname='NA',
               nname='CL',
               neutral=True,
               input='SOL'
              )

Will try to add 3 NA ions and 0 CL ions.
Select a continuous group of solvent molecules
Selected 13: 'SOL'

Processing topology
Replacing 3 solute molecules in topology file (topol.top)  by 3 NA and 0 CL ions.


                       :-) GROMACS - gmx genion, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx genion -s ions.tpr -o structures/solvate_ions.gro -p topol.top -pname NA -nname CL -neutral

Reading file ions.tpr, VERSION 2023 (single precision)
Reading file ions.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 30910 elements
Group     1 (        Protein) has  3445 elements
Group     2 (      Protein-H) has  1769 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1116 elements
Group     7 (    MainChain+H) has  1134 elements
Group     8 (      SideChain) has  2311 elements
Group     9 (    SideChain-H) has   859 elements
Group    10 (    Prot-Masses) has  3445 element

(0, None, None)

## Adding CDR coordinates to index

For the subsequent analysis of the trajectories, we need the coordinates of the C-alpha atoms of the CDR residues, so we first write in the index file which of the atoms are part of the CDRs.

In [11]:
input_for_make_ndx = (
    '"C-alpha" & ri 26-32',
    '"C-alpha" & ri 52-59',
    '"C-alpha" & ri 101-109',
    '"C-alpha" & ri 144-154',
    '"C-alpha" & ri 170-176',
    '"C-alpha" & ri 209-217',
    'name 17 CDR-H1',
    'name 18 CDR-H2',
    'name 19 CDR-H3',
    'name 20 CDR-L1',
    'name 21 CDR-L2',
    'name 22 CDR-L3',
    '17 | 18 | 19 | 20 | 21 | 22',
    'name 23 CDRs',
    'q',
)

In [12]:
gromacs.make_ndx(f='structures/solvate_ions.gro',
                 input=input_for_make_ndx
                )

Going to read 0 old index file(s)
Analysing residue names:
There are:   227    Protein residues
There are:  9152      Water residues
There are:     3        Ion residues
Analysing Protein...

  0 System              : 30904 atoms
  1 Protein             :  3445 atoms
  2 Protein-H           :  1769 atoms
  3 C-alpha             :   227 atoms
  4 Backbone            :   681 atoms
  5 MainChain           :   910 atoms
  6 MainChain+Cb        :  1116 atoms
  7 MainChain+H         :  1134 atoms
  8 SideChain           :  2311 atoms
  9 SideChain-H         :   859 atoms
 10 Prot-Masses         :  3445 atoms
 11 non-Protein         : 27459 atoms
 12 Water               : 27456 atoms
 13 SOL                 : 27456 atoms
 14 non-Water           :  3448 atoms
 15 Ion                 :     3 atoms
 16 Water_and_ions      : 27459 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't

                      :-) GROMACS - gmx make_ndx, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx make_ndx -f structures/solvate_ions.gro


Reading structure file

GROMACS reminds you: "This really is a pretty scene, could you ask your kid to smile please?" (Joe Jackson)



(0, None, None)

## Energy minimization

Here we perform energy minimization to ensure that the system has no steric clashes or inappropriate geometry. After that, one can check plot of potential energy of system during minimization. It can be found in `plots/` directory.

In [13]:
os.mkdir('em')

In [14]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'em.mdp'),
               c='structures/solvate_ions.gro',
               p='topol.top',
               n='index.ndx',
               o='em/em.tpr'
              )

Setting the LD random seed to -658538809

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'NA'

The largest distance between excluded atoms is 0.449 nm between atom 2179 and 3147
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 64x64x64, spacing 0.120 0.120 0.120

Estimate for the relative computational load of the PME mesh part: 0.28

This run will generate roughly 2 Mb of data


                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/em.mdp -c structures/solvate_ions.gro -p topol.top -n index.ndx -o em/em.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/em.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group rest is 65253.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

There was 1 NOTE

GROMACS reminds you: "Not

(0, None, None)

In [15]:
gromacs.mdrun(deffnm='em/em')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx mdrun -deffnm em/em -v

Reading file em/em.tpr, VERSION 2023 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 12 OpenMP threads 


Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+02
   Number of steps    =        10000
Step=    0, Dmax= 1.0e-02 nm, Epot= -2.07937e+05 Fmax= 1.53642e+05, atom= 1854
Step=    1, Dmax= 1.0e-02 nm, Epot= -2.31550e+05 Fmax= 5.71993e+04, atom= 1851
Step=    2, Dmax= 1.2e-02 nm, Epot= -2.62147e+05 Fmax= 2.33147e+04, atom= 8402
Step=    3, Dmax= 1.4e-02 nm, Epot= -2.96818e+05 Fmax= 9.

Step=  159, Dmax= 2.2e-02 nm, Epot= -4.28681e+05 Fmax= 1.81898e+04, atom= 1838
Step=  161, Dmax= 1.3e-02 nm, Epot= -4.29051e+05 Fmax= 2.00821e+03, atom= 1838
Step=  162, Dmax= 1.6e-02 nm, Epot= -4.29302e+05 Fmax= 2.28343e+04, atom= 1838
Step=  163, Dmax= 1.9e-02 nm, Epot= -4.29825e+05 Fmax= 6.42724e+03, atom= 1838
Step=  165, Dmax= 1.1e-02 nm, Epot= -4.29905e+05 Fmax= 1.11958e+04, atom= 1838
Step=  166, Dmax= 1.4e-02 nm, Epot= -4.30040e+05 Fmax= 9.75171e+03, atom= 1838
Step=  167, Dmax= 1.6e-02 nm, Epot= -4.30049e+05 Fmax= 1.55486e+04, atom= 1838
Step=  168, Dmax= 2.0e-02 nm, Epot= -4.30182e+05 Fmax= 1.45882e+04, atom= 1838
Step=  170, Dmax= 1.2e-02 nm, Epot= -4.30444e+05 Fmax= 3.58444e+03, atom= 1838
Step=  171, Dmax= 1.4e-02 nm, Epot= -4.30480e+05 Fmax= 1.79721e+04, atom= 1838
Step=  172, Dmax= 1.7e-02 nm, Epot= -4.30810e+05 Fmax= 8.22671e+03, atom= 1838
Step=  174, Dmax= 1.0e-02 nm, Epot= -4.30923e+05 Fmax= 7.37454e+03, atom= 1838
Step=  175, Dmax= 1.2e-02 nm, Epot= -4.30984e+05 Fma

Step=  329, Dmax= 1.3e-02 nm, Epot= -4.41843e+05 Fmax= 1.26822e+04, atom= 1660
Step=  330, Dmax= 1.6e-02 nm, Epot= -4.41911e+05 Fmax= 1.13945e+04, atom= 1660
Step=  332, Dmax= 9.4e-03 nm, Epot= -4.42022e+05 Fmax= 3.15788e+03, atom= 1660
Step=  333, Dmax= 1.1e-02 nm, Epot= -4.42044e+05 Fmax= 1.40644e+04, atom= 1660
Step=  334, Dmax= 1.4e-02 nm, Epot= -4.42179e+05 Fmax= 6.90772e+03, atom= 1660
Step=  336, Dmax= 8.1e-03 nm, Epot= -4.42235e+05 Fmax= 5.55543e+03, atom= 1660
Step=  337, Dmax= 9.8e-03 nm, Epot= -4.42267e+05 Fmax= 9.58933e+03, atom= 1660
Step=  338, Dmax= 1.2e-02 nm, Epot= -4.42328e+05 Fmax= 8.38169e+03, atom= 1660
Step=  340, Dmax= 7.0e-03 nm, Epot= -4.42401e+05 Fmax= 2.50085e+03, atom= 1660
Step=  341, Dmax= 8.4e-03 nm, Epot= -4.42469e+05 Fmax= 1.03138e+04, atom= 1660
Step=  342, Dmax= 1.0e-02 nm, Epot= -4.42557e+05 Fmax= 5.35916e+03, atom= 1660
Step=  344, Dmax= 6.1e-03 nm, Epot= -4.42608e+05 Fmax= 3.93480e+03, atom= 1660
Step=  345, Dmax= 7.3e-03 nm, Epot= -4.42655e+05 Fma

Step=  480, Dmax= 8.1e-03 nm, Epot= -4.48416e+05 Fmax= 5.55161e+03, atom= 1660
Step=  481, Dmax= 9.7e-03 nm, Epot= -4.48431e+05 Fmax= 9.41919e+03, atom= 1660
Step=  482, Dmax= 1.2e-02 nm, Epot= -4.48471e+05 Fmax= 8.38240e+03, atom= 1660
Step=  484, Dmax= 7.0e-03 nm, Epot= -4.48527e+05 Fmax= 2.37597e+03, atom= 1660
Step=  485, Dmax= 8.4e-03 nm, Epot= -4.48563e+05 Fmax= 1.03426e+04, atom= 1660
Step=  486, Dmax= 1.0e-02 nm, Epot= -4.48631e+05 Fmax= 5.15184e+03, atom= 1660
Step=  488, Dmax= 6.0e-03 nm, Epot= -4.48667e+05 Fmax= 4.05856e+03, atom= 1660
Step=  489, Dmax= 7.2e-03 nm, Epot= -4.48693e+05 Fmax= 7.13608e+03, atom= 1660
Step=  490, Dmax= 8.7e-03 nm, Epot= -4.48731e+05 Fmax= 6.14523e+03, atom= 1660
Step=  491, Dmax= 1.0e-02 nm, Epot= -4.48741e+05 Fmax= 9.93605e+03, atom= 1660
Step=  492, Dmax= 1.2e-02 nm, Epot= -4.48781e+05 Fmax= 9.19795e+03, atom= 1660
Step=  494, Dmax= 7.5e-03 nm, Epot= -4.48842e+05 Fmax= 2.36058e+03, atom= 1660
Step=  495, Dmax= 9.0e-03 nm, Epot= -4.48873e+05 Fma

Step=  646, Dmax= 5.5e-03 nm, Epot= -4.53217e+05 Fmax= 1.78180e+03, atom= 1660
Step=  647, Dmax= 6.6e-03 nm, Epot= -4.53251e+05 Fmax= 8.26774e+03, atom= 1660
Step=  648, Dmax= 8.0e-03 nm, Epot= -4.53295e+05 Fmax= 4.02006e+03, atom= 1660
Step=  650, Dmax= 4.8e-03 nm, Epot= -4.53319e+05 Fmax= 3.27872e+03, atom= 1660
Step=  651, Dmax= 5.7e-03 nm, Epot= -4.53340e+05 Fmax= 5.60366e+03, atom= 1660
Step=  652, Dmax= 6.9e-03 nm, Epot= -4.53366e+05 Fmax= 4.92571e+03, atom= 1660
Step=  653, Dmax= 8.3e-03 nm, Epot= -4.53377e+05 Fmax= 7.83059e+03, atom= 1660
Step=  654, Dmax= 9.9e-03 nm, Epot= -4.53402e+05 Fmax= 7.34599e+03, atom= 1660
Step=  656, Dmax= 6.0e-03 nm, Epot= -4.53444e+05 Fmax= 1.82436e+03, atom= 1660
Step=  657, Dmax= 7.1e-03 nm, Epot= -4.53473e+05 Fmax= 8.99628e+03, atom= 1660
Step=  658, Dmax= 8.6e-03 nm, Epot= -4.53523e+05 Fmax= 4.20518e+03, atom= 1660
Step=  660, Dmax= 5.1e-03 nm, Epot= -4.53547e+05 Fmax= 3.64676e+03, atom= 1660
Step=  661, Dmax= 6.2e-03 nm, Epot= -4.53566e+05 Fma

...

Step= 5409, Dmax= 2.2e-03 nm, Epot= -4.79351e+05 Fmax= 1.33766e+03, atom= 2085
Step= 5411, Dmax= 1.3e-03 nm, Epot= -4.79352e+05 Fmax= 6.53360e+02, atom= 2085
Step= 5412, Dmax= 1.6e-03 nm, Epot= -4.79353e+05 Fmax= 1.72651e+03, atom= 2085
Step= 5413, Dmax= 1.9e-03 nm, Epot= -4.79355e+05 Fmax= 1.13993e+03, atom= 2085
Step= 5415, Dmax= 1.1e-03 nm, Epot= -4.79356e+05 Fmax= 5.75553e+02, atom= 2085
Step= 5416, Dmax= 1.3e-03 nm, Epot= -4.79357e+05 Fmax= 1.49249e+03, atom= 2085
Step= 5417, Dmax= 1.6e-03 nm, Epot= -4.79360e+05 Fmax= 9.78558e+02, atom= 2085
Step= 5418, Dmax= 1.9e-03 nm, Epot= -4.79361e+05 Fmax= 1.99585e+03, atom= 2085
Step= 5419, Dmax= 2.3e-03 nm, Epot= -4.79363e+05 Fmax= 1.56364e+03, atom= 2085
Step= 5421, Dmax= 1.4e-03 nm, Epot= -4.79364e+05 Fmax= 5.76305e+02, atom= 2085
Step= 5422, Dmax= 1.7e-03 nm, Epot= -4.79366e+05 Fmax= 1.98096e+03, atom= 2085
Step= 5423, Dmax= 2.0e-03 nm, Epot= -4.79368e+05 Fmax= 1.10020e+03, atom= 2085
Step= 5425, Dmax= 1.2e-03 nm, Epot= -4.79369e+05 Fma

Step= 5545, Dmax= 2.9e-03 nm, Epot= -4.79547e+05 Fmax= 1.02782e+03, atom= 2085
Step= 5547, Dmax= 1.7e-03 nm, Epot= -4.79549e+05 Fmax= 1.61475e+03, atom= 2085
Step= 5548, Dmax= 2.1e-03 nm, Epot= -4.79550e+05 Fmax= 1.54727e+03, atom= 2085
Step= 5549, Dmax= 2.5e-03 nm, Epot= -4.79550e+05 Fmax= 2.25569e+03, atom= 2085
Step= 5551, Dmax= 1.5e-03 nm, Epot= -4.79553e+05 Fmax= 2.12051e+02, atom= 24253
Step= 5552, Dmax= 1.8e-03 nm, Epot= -4.79562e+05 Fmax= 2.80784e+02, atom= 2085
Step= 5553, Dmax= 2.1e-03 nm, Epot= -4.79565e+05 Fmax= 2.96924e+03, atom= 2085
Step= 5554, Dmax= 2.6e-03 nm, Epot= -4.79570e+05 Fmax= 9.69091e+02, atom= 2085
Step= 5556, Dmax= 1.5e-03 nm, Epot= -4.79571e+05 Fmax= 1.38970e+03, atom= 2085
Step= 5557, Dmax= 1.8e-03 nm, Epot= -4.79573e+05 Fmax= 1.44946e+03, atom= 2085
Step= 5558, Dmax= 2.2e-03 nm, Epot= -4.79573e+05 Fmax= 1.94915e+03, atom= 2085
Step= 5559, Dmax= 2.7e-03 nm, Epot= -4.79573e+05 Fmax= 2.13792e+03, atom= 2085
Step= 5560, Dmax= 3.2e-03 nm, Epot= -4.79574e+05 Fm

Step= 5682, Dmax= 1.9e-03 nm, Epot= -4.79747e+05 Fmax= 1.74054e+03, atom= 2085
Step= 5683, Dmax= 2.3e-03 nm, Epot= -4.79748e+05 Fmax= 1.76236e+03, atom= 2085
Step= 5685, Dmax= 1.4e-03 nm, Epot= -4.79751e+05 Fmax= 3.35174e+02, atom= 2085
Step= 5686, Dmax= 1.6e-03 nm, Epot= -4.79752e+05 Fmax= 2.19749e+03, atom= 2085
Step= 5687, Dmax= 2.0e-03 nm, Epot= -4.79755e+05 Fmax= 8.23545e+02, atom= 2085
Step= 5689, Dmax= 1.2e-03 nm, Epot= -4.79755e+05 Fmax= 9.94391e+02, atom= 2085
Step= 5690, Dmax= 1.4e-03 nm, Epot= -4.79757e+05 Fmax= 1.17870e+03, atom= 2085
Step= 5691, Dmax= 1.7e-03 nm, Epot= -4.79759e+05 Fmax= 1.43751e+03, atom= 2085
Step= 5692, Dmax= 2.0e-03 nm, Epot= -4.79760e+05 Fmax= 1.69326e+03, atom= 2085
Step= 5693, Dmax= 2.4e-03 nm, Epot= -4.79760e+05 Fmax= 2.07219e+03, atom= 2085
Step= 5694, Dmax= 2.9e-03 nm, Epot= -4.79761e+05 Fmax= 2.43766e+03, atom= 2085
Step= 5696, Dmax= 1.8e-03 nm, Epot= -4.79764e+05 Fmax= 2.72690e+02, atom= 2085
Step= 5697, Dmax= 2.1e-03 nm, Epot= -4.79768e+05 Fma

Step= 5816, Dmax= 1.7e-03 nm, Epot= -4.79919e+05 Fmax= 1.30875e+03, atom= 2085
Step= 5817, Dmax= 2.1e-03 nm, Epot= -4.79920e+05 Fmax= 1.91909e+03, atom= 2085
Step= 5818, Dmax= 2.5e-03 nm, Epot= -4.79921e+05 Fmax= 1.94578e+03, atom= 2085
Step= 5820, Dmax= 1.5e-03 nm, Epot= -4.79923e+05 Fmax= 3.77283e+02, atom= 2085
Step= 5821, Dmax= 1.8e-03 nm, Epot= -4.79924e+05 Fmax= 2.39567e+03, atom= 2085
Step= 5822, Dmax= 2.2e-03 nm, Epot= -4.79928e+05 Fmax= 9.49164e+02, atom= 2085
Step= 5824, Dmax= 1.3e-03 nm, Epot= -4.79929e+05 Fmax= 1.05256e+03, atom= 2085
Step= 5825, Dmax= 1.6e-03 nm, Epot= -4.79930e+05 Fmax= 1.35777e+03, atom= 2085
Step= 5826, Dmax= 1.9e-03 nm, Epot= -4.79931e+05 Fmax= 1.52620e+03, atom= 2085
Step= 5827, Dmax= 2.3e-03 nm, Epot= -4.79932e+05 Fmax= 1.94312e+03, atom= 2085
Step= 5828, Dmax= 2.7e-03 nm, Epot= -4.79933e+05 Fmax= 2.21109e+03, atom= 2085
Step= 5830, Dmax= 1.6e-03 nm, Epot= -4.79936e+05 Fmax= 2.85658e+02, atom= 2085
Step= 5831, Dmax= 1.9e-03 nm, Epot= -4.79937e+05 Fma

(0, None, None)

In [17]:
if not os.path.exists('plots/'):
    os.mkdir('plots')

gromacs.energy(f='em/em.edr',
               o='plots/potential.xvg',
               input=('Potential', '0')
              )


Statistics over 5943 steps [ 0.0000 through 5942.0000 ps ], 1 data sets
All statistics are over 4703 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -468675       6200    15982.1     -40240  (kJ/mol)


                       :-) GROMACS - gmx energy, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx energy -f em/em.edr -o plots/potential.xvg

Opened em/em.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Coulomb-(SR)  
  9  Coul.-recip.    10  Potential       11  Pressure        12  Vir-XX        
 13  Vir-XY          14  Vir-XZ          15  Vir-YX          16  Vir-YY        
 17  Vir-YZ          18  Vir-ZX          19  Vir-ZY          20  Vir-ZZ        
 21  Pre

(0, None, None)

## NVT equilibration

In [18]:
os.mkdir('nvt')

In [19]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'nvt.mdp'),
               c='em/em.gro',
               p='topol.top',
               n='index.ndx',
               o='nvt/nvt.tpr'
              )

Setting the LD random seed to -126419009

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Setting gen_seed to -210224258

Velocities were taken from a Maxwell distribution at 310 K

The largest distance between excluded atoms is 0.452 nm between atom 2179 and 3147

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.034 nm, buffer size 0.034 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list 

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/nvt.mdp -c em/em.gro -p topol.top -n index.ndx -o nvt/nvt.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8658.59
Number of degrees of freedom in T-Coupling group non-Protein is 54918.41

GROMACS reminds you: "There is no place like ~" (Anonymous)



(0, None, None)

In [20]:
gromacs.mdrun(deffnm='nvt/nvt')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx mdrun -deffnm nvt/nvt

Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.155

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 

starting mdrun 'Protein in water'
1000000 steps,   2000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     3864.770      644.129      600.0
                 (ns/day)    (hour/ns)
Performance:      268.269        0.089

GROMACS remind

(0, None, None)

In [21]:
if not os.path.exists('plots/'):
    os.mkdir('plots')

gromacs.energy(f='nvt/nvt.edr',
               o='plots/temperature.xvg',
               input=('Temperature', '0')
              )


Statistics over 1000001 steps [ 0.0000 through 2000.0000 ps ], 1 data sets
All statistics are over 10001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 309.997      0.031    2.15547 -0.0310179  (K)


                       :-) GROMACS - gmx energy, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx energy -f nvt/nvt.edr -o plots/temperature.xvg

Opened nvt/nvt.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Vir-XX          20  Vir-XY        
 2

(0, None, None)

In [22]:
gromacs.trjconv(s='nvt/nvt.tpr',
                f='nvt/nvt.trr',
                n='index.ndx',
                o='nvt/nvt_noPBC.trr',
                pbc='mol',
                input=('System')
               )

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx trjconv -s nvt/nvt.tpr -f nvt/nvt.trr -n index.ndx -o nvt/nvt_noPBC.trr -pbc mol

Will write trr: Trajectory in portable xdr format
Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Reading file nvt/nvt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 30904 elements
Group     1 (        Protein) has  3445 elements
Group     2 (      Protein-H) has  1769 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1116 elements
Group     7 (    MainChain+H) has  1134 elements
Group     8 (      SideChain) has  2311 elements
Group     9 (    SideChain-H) has   859 elements
Gr

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


Last frame        200 time 2000.000   
 ->  frame    200 time 2000.000      
Last written: frame    200 time 2000.000


GROMACS reminds you: "Your Excellency, I have no need of this hypothesis." (Pierre Laplace, to Napoleon on why his works on celestial mechanics make no mention of God.)



(0, None, None)

## NPT equilibration

In [23]:
os.mkdir('npt')

In [24]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'npt.mdp'),
               c='nvt/nvt.gro',
               t='nvt/nvt.cpt',
               p='topol.top',
               n='index.ndx',
               o='npt/npt.tpr'
              )

Setting the LD random seed to 2143289275

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

The largest distance between excluded atoms is 0.463 nm between atom 2179 and 3147

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.034 nm, buffer size 0.034 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole tra

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/npt.mdp -c nvt/nvt.gro -t nvt/nvt.cpt -p topol.top -n index.ndx -o npt/npt.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8658.59
Number of degrees of freedom in T-Coupling group non-Protein is 54918.41
Last frame         -1 time 2000.000   

GROMACS reminds you: "I Don't Want to Catch Anybody Not Drinking." (Monty Python)



(0, None, None)

In [25]:
gromacs.mdrun(deffnm='npt/npt')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx mdrun -deffnm npt/npt

Reading file npt/npt.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.155

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 

starting mdrun 'Protein in water'
1000000 steps,   2000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     3962.180      660.364      600.0
                 (ns/day)    (hour/ns)
Performance:      261.674        0.092

GROMACS remind

(0, None, None)

In [26]:
if not os.path.exists('plots/'):
    os.mkdir('plots')

gromacs.energy(f='npt/npt.edr',
               o='plots/pressure.xvg',
               input=('Pressure', '0')
              )

gromacs.energy(f='npt/npt.edr',
               o='plots/density.xvg',
               input=('Density', '0')
              )


Statistics over 1000001 steps [ 0.0000 through 2000.0000 ps ], 1 data sets
All statistics are over 10001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                  -0.611404       0.86    198.096   0.677613  (bar)

Statistics over 1000001 steps [ 0.0000 through 2000.0000 ps ], 1 data sets
All statistics are over 10001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Density                     1016.29       0.09    6.00342   0.553049  (kg/m^3)


                       :-) GROMACS - gmx energy, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx energy -f npt/npt.edr -o plots/pressure.xvg

Opened npt/npt.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  

(0, None, None)

In [27]:
gromacs.trjconv(s='npt/npt.tpr',
                f='npt/npt.trr',
                n='index.ndx',
                o='npt/npt_noPBC.trr',
                pbc='mol',
                input=('System')
               )

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx trjconv -s npt/npt.tpr -f npt/npt.trr -n index.ndx -o npt/npt_noPBC.trr -pbc mol

Will write trr: Trajectory in portable xdr format
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Reading file npt/npt.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 30904 elements
Group     1 (        Protein) has  3445 elements
Group     2 (      Protein-H) has  1769 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1116 elements
Group     7 (    MainChain+H) has  1134 elements
Group     8 (      SideChain) has  2311 elements
Group     9 (    SideChain-H) has   859 elements
Gr

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


Last frame        200 time 2000.000   
 ->  frame    200 time 2000.000      
Last written: frame    200 time 2000.000


GROMACS reminds you: "Strength is just an accident arising from the weakness of others" (Joseph Conrad)



(0, None, None)

## Production run

### Short test production run

In [28]:
os.mkdir('prod_test')

In [29]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'production_short.mdp'),
               c='npt/npt.gro',
               t='npt/npt.cpt',
               p='topol.top',
               n='index.ndx',
               o='prod_test/prod_test.tpr'
              )

Setting the LD random seed to -9580562

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

The largest distance between excluded atoms is 0.452 nm between atom 299 and 1506

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.035 nm, buffer size 0.035 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole trajec

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/production_short.mdp -c npt/npt.gro -t npt/npt.cpt -p topol.top -n index.ndx -o prod_test/prod_test.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8658.59
Number of degrees of freedom in T-Coupling group non-Protein is 54918.41
Last frame         -1 time 2000.000   

GROMACS reminds you: "I Am a Poor Lonesome Cowboy" (Lucky Luke)



(0, None, None)

In [30]:
gromacs.mdrun(deffnm='prod_test/prod_test')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx mdrun -deffnm prod_test/prod_test

Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.157

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 

starting mdrun 'Protein in water'
500000 steps,   1000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     1983.922      330.655      600.0
                 (ns/day)    (hour/ns)
Performance:      261.300      

(0, None, None)

In [31]:
gromacs.trjconv(s='prod_test/prod_test.tpr',
                f='prod_test/prod_test.xtc',
                n='index.ndx',
                o='prod_test/prod_test_noPBC.xtc',
                pbc='mol',
                input=('Protein')
               )

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/remicade
Command line:
  gmx trjconv -s prod_test/prod_test.tpr -f prod_test/prod_test.xtc -n index.ndx -o prod_test/prod_test_noPBC.xtc -pbc mol

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Reading file prod_test/prod_test.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 30904 elements
Group     1 (        Protein) has  3445 elements
Group     2 (      Protein-H) has  1769 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1116 elements
Group     7 (    MainChain+H) has  1134 elements
Group     8 (      SideCh

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'


Last frame       5000 time 1000.000   
 ->  frame   5000 time 1000.000      
Last written: frame   5000 time 1000.000


GROMACS reminds you: "If you want to save your child from polio, you can pray or you can inoculate... choose science." (Carl Sagan)



(0, None, None)

### Main production run

In [54]:
os.mkdir('prod')

In [56]:
gromacs.grompp(f=os.path.join(MDP_FILES_DIRECTORY, 'production.mdp'),
               c='npt/npt.gro',
               t='npt/npt.cpt',
               p='topol.top',
               n='index.ndx',
               o='prod/prod.tpr'
              )

Setting the LD random seed to -68435985

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_H'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

The largest distance between excluded atoms is 0.431 nm between atom 1288 and 1296

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.035 nm, buffer size 0.035 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole traj

                       :-) GROMACS - gmx grompp, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx grompp -f /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/production.mdp -c npt/npt.gro -t npt/npt.cpt -p topol.top -n index.ndx -o prod/prod.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group Protein is 8724.63
Number of degrees of freedom in T-Coupling group non-Protein is 61500.37
Last frame         -1 time 2000.000   

NOTE 1 [file /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/mdp_files/production.mdp]:
  This run will generate roughly 6565 Mb of data


There was 1 NOTE

GROMACS reminds you: "A Man Needs a Maid" (N. Young)



(0, None, None)

In [57]:
gromacs.mdrun(deffnm='prod/prod')

                       :-) GROMACS - gmx mdrun, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx mdrun -deffnm prod/prod

Reading file prod/prod.tpr, VERSION 2023 (single precision)
Changing nstlist from 10 to 100, rlist from 1 to 1.157

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 6 OpenMP threads 



starting mdrun 'Protein in water'
40000000 steps,  80000.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:   183486.274    30581.047      600.0
                         8h29:41
                 (ns/day)    (hour/ns)
Performanc

(0, None, None)

In [58]:
gromacs.trjconv(s='prod/prod.tpr',
                f='prod/prod.xtc',
                n='index.ndx',
                o='prod/prod_noPBC.xtc',
                pbc='mol',
                input=('Protein')
               )

                      :-) GROMACS - gmx trjconv, 2023 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/nikitos/Bioinformatics/BI/BIOCAD_project/md_simulations/remicade_humanization/candidate_Aa
Command line:
  gmx trjconv -s prod/prod.tpr -f prod/prod.xtc -n index.ndx -o prod/prod_noPBC.xtc -pbc mol -center

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod/prod.tpr, VERSION 2023 (single precision)
Reading file prod/prod.tpr, VERSION 2023 (single precision)
Group     0 (         System) has 34227 elements
Group     1 (        Protein) has  3476 elements
Group     2 (      Protein-H) has  1773 elements
Group     3 (        C-alpha) has   227 elements
Group     4 (       Backbone) has   681 elements
Group     5 (      MainChain) has   910 elements
Group     6 (   MainChain+Cb) has  1117 elements
Group     7 (    MainChain+H) has  1133 elements
Group     8 (      SideChain) has  2343 elements
Group     9 ( 

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Last frame      40000 time 80000.000   
 ->  frame  40000 time 80000.000      
Last written: frame  40000 time 80000.000


GROMACS reminds you: "How wonderful that we have met with a paradox. Now we have some hope of making progress." (Niels Bohr)



(0, None, None)